In [ ]:
# Import libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# read datasets
df_credits = pd.read_csv("archive/tmdb_5000_credits.csv")
df_movies = pd.read_csv("archive/tmdb_5000_movies.csv")

Load in the datasets and check for first five rows

In [ ]:
df_credits.head()

In [ ]:
df_movies.head()

### Check the general structure of both the datasets

In [ ]:
# Check the shape of the first dataset
df_credits.shape

In [ ]:
# Check the shape of the second dataset
df_movies.shape

**df_credits has 4803 rows and 4 columns while df_mov has 4803 rows and 20 columns. Let's explore further**

In [ ]:
# Check for the duplicates in the first dataset
sum(df_credits.duplicated())

In [ ]:
# Check for the duplicates in the second dataset
sum(df_movies.duplicated())

**Both the datasets do not have any duplicate rows**

checking data type of datasets

In [ ]:
# Check for the datatypes of each variable in the first dataset
df_credits.dtypes

In [ ]:
# Check for the datatypes of each variable in the second dataset
df_movies.dtypes

`release_date` variable should be of the form datetime 

In [ ]:
# Check for unique values in each variable in the first dataset
df_credits.nunique()

**While there are 4803 movies in total, there are only 4800 unique titles which suggest the presence of duplicates**


In [ ]:
# Check for unique values in each variable in the second dataset
df_movies.nunique()

1. While there are 4803 movies in total, there are only 4800 unique titles and which suggest the presence of duplicates
2. There are 4800 titles but 4801 original titles which suggests further investigation in the 2 variables
3. There are 4802 unique popularity values instead of 4803 which suggests presence of duplicates
4. Status has only 3 unique values which needs further investigation to find if we need all the observations or some of them can be removed

In [ ]:
# Check for detailed info in the first dataset
df_credits.info()

In [ ]:
# Check for detailed info in the second dataset
df_movies.info()

**Presence of null values can be seen in some variables namely - `homepage`, `overview`, `release_date`, `runtime`, and `tagline`**

In [ ]:
df_movies.status.value_counts()

**It can be seen that there are almost all the obseravtions which belong to `Released` status. Therefore, we need to limit the dataset to this value of status only as including other status types also makes no sense**


In [ ]:
df_movies.isnull().sum()

**There are huge number of null counts in `homepage` followed by `tagline`. These are the variables which we might not even require in our further analysis**

##### As we saw above `original_title` and `title` unique values did not match, so we need to explore them further


In [ ]:
# Check the instances where original title do not match with title column
df_movies[['original_title', 'title']].query('original_title != title')

**So, we saw that there are 261 instances where `original_title` doesn't match with `title`. So, it's possible that number of unique values differ in both the columns. Therefore, we are on the same place as before that we need to explore `title` column further which we will do in the _Further Exploration and Cleaning_ section**

### Further Exploration and Cleaning 

#### Issues to be resolved:
1. Filter df_movies to the observations with Released status in the new dataset cl_mov which is always a good decision because we might need the original dataset at any time in the analysis

2. Drop the unnecessary columns

3. Drop the null observations in any variable

4. Explore and check title variable further for duplicates in df_movies

5. change the data type of the `release_date` to datetime in df_movies

6. Try to resolve the variables that find difficult to explore

**1. Filter df_mov to the observations with _Released_ status in the new dataset cl_mov which is always a good decision because we might need the original dataset at any time in the analysis**

In [ ]:
# Filter df_movies with status 'Released'
cl_mov = df_movies[df_movies['status'] == "Released"]
cl_mov.head(2)

In [ ]:
# confirm only status is "Released"
cl_mov.status.unique()

**2. Drop the unnecessary columns**

In [ ]:
# Drop columns not required further for analysis
cl_mov.drop(['homepage', 'keywords', 'original_language', 'original_title', 'tagline', 'overview', 'spoken_languages', 'status'], axis=1, inplace = True)
cl_mov.head(2)

In [ ]:
# Check the shape again of cl_mov
cl_mov.shape

In [ ]:
# Check the null counts again of cl_mov
cl_mov.isnull().sum()